In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn import tree
from IPython.display import Image
import pydotplus
import graphviz
import time
%matplotlib inline

In [114]:
voice = pd.read_csv(r"C:\Users\nstow\Desktop\Python\voice.csv")

## check for Null or NAN values
voice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
meanfreq    3168 non-null float64
sd          3168 non-null float64
median      3168 non-null float64
Q25         3168 non-null float64
Q75         3168 non-null float64
IQR         3168 non-null float64
skew        3168 non-null float64
kurt        3168 non-null float64
sp.ent      3168 non-null float64
sfm         3168 non-null float64
mode        3168 non-null float64
centroid    3168 non-null float64
meanfun     3168 non-null float64
minfun      3168 non-null float64
maxfun      3168 non-null float64
meandom     3168 non-null float64
mindom      3168 non-null float64
maxdom      3168 non-null float64
dfrange     3168 non-null float64
modindx     3168 non-null float64
label       3168 non-null object
dtypes: float64(20), object(1)
memory usage: 519.8+ KB


In [115]:
## Ready the data to be used in the decision tree

voice_label = pd.DataFrame()
voice_label['label'] = voice['label']
voice = voice.drop(columns = 'label')

voice_label['label_male'] = np.where(voice_label['label'] == 'male', 1, 0)
voice_label = voice_label.drop(columns = 'label')

## Single Decision Tree Test

In [116]:
## Create a single dicision tree and determine the accuracy

dTree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=1,
    max_depth=5,
    random_state = 1337
)
dTree.fit(voice, voice_label)

# Render our tree.
dot_data = tree.export_graphviz(
    dTree, out_file=None,
    feature_names=voice.columns,
    class_names=['Male', 'Female'],
    filled=True
)

print(dTree.score(voice, voice_label, sample_weight=None))

0.9071969696969697


In [117]:
## compare accuracy

dTree1 = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=5,
    max_depth=8,
    random_state = 1337
)
dTree1.fit(voice, voice_label)

# Render our tree.
dot_data = tree.export_graphviz(
    dTree1, out_file=None,
    feature_names=voice.columns,
    class_names=['Male', 'Female'],
    filled=True
)

print(dTree1.score(voice, voice_label, sample_weight=None))

0.9958964646464646


In [118]:
dTree2 = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=6,
    max_depth=9,
    random_state = 1337
)
dTree2.fit(voice, voice_label)

# Render our tree.
dot_data = tree.export_graphviz(
    dTree2, out_file=None,
    feature_names=voice.columns,
    class_names=['Male', 'Female'],
    filled=True
)

print(dTree2.score(voice, voice_label, sample_weight=None))

0.9943181818181818


In [120]:
start_time = time.time()

dTree3 = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=6,
    max_depth=111,
    random_state = 1337
)
dTree3.fit(voice, voice_label)

# Render our tree.
dot_data = tree.export_graphviz(
    dTree3, out_file=None,
    feature_names=voice.columns,
    class_names=['Male', 'Female'],
    filled=True
)
print("--- %s seconds ---" % (time.time() - start_time))
print(dTree3.score(voice, voice_label, sample_weight=None))

--- 0.0319674015045166 seconds ---
1.0


### We have achieved 100% accuracy, or the 'best' decision tree that can be achieved for this data set. The algorithm runs in about 3/100th of a second.

## Random Forest Test

In [121]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [154]:
start_time1 = time.time()

voice, voice_label = make_classification(n_samples=1000, 
                            n_features=4,
                            n_informative=2)
clf = RandomForestClassifier(n_estimators=100, max_depth=2)

clf.fit(voice, voice_label)

print("--- %s seconds ---" % (time.time() - start_time1))
print(clf.score(voice, voice_label))

--- 0.06905221939086914 seconds ---
0.936


In [148]:
start_time2 = time.time()

voice, voice_label = make_classification(n_samples=100, 
                            n_features=4,
                            n_informative=2)
clf1 = RandomForestClassifier(n_estimators=100, max_depth=2)

clf1.fit(voice, voice_label)

print("--- %s seconds ---" % (time.time() - start_time2))
print(clf1.score(voice, voice_label))

--- 0.04989814758300781 seconds ---
0.93


In [128]:
start_time3 = time.time()

voice, voice_label = make_classification(n_samples=100, 
                            n_features=4,
                            n_informative=2)
clf2 = RandomForestClassifier(n_estimators=10, max_depth=2)

clf2.fit(voice, voice_label)

print("--- %s seconds ---" % (time.time() - start_time3))
print(clf2.score(voice, voice_label))

--- 0.007978677749633789 seconds ---
0.92


In [130]:
start_time4 = time.time()

voice, voice_label = make_classification(n_samples=10, 
                            n_features=4,
                            n_informative=2)
clf3 = RandomForestClassifier(n_estimators=1, max_depth=2)

clf3.fit(voice, voice_label)

print("--- %s seconds ---" % (time.time() - start_time4))
print(clf3.score(voice, voice_label))

--- 0.001996755599975586 seconds ---
0.8


### We got the time of the Random Tree Classifier below that of the single Decision Tree but the accuracy is at 80%. So the single tree is accurate all of the time, and the Random Tree is only accurate 4 out of 5 times. To get close to the same accuracy with the Random Tree Classifier, we have to more than double the time.